# Prendre en main LangChain pas à pas

Objectif de ce notebook :  
- Comprendre les **briques de base** de LangChain (LLM, prompt, chain).  
- Visualiser un **schéma simple** de pipeline.  
- Construire un mini **pipeline de question/réponse**, puis un mini **pipeline RAG simplifié**.

> Remarque : ce notebook utilise un **LLM local** (Ollama ou LM Studio) configuré via le fichier `.env`.

## Les briques de base de LangChain

On va manipuler 3 briques principales :

1. **LLM** : le modèle de langage (par ex. `gpt-4o-mini`)  
2. **Prompt** : la manière dont on parle au modèle (system + user + variables)  
3. **Chain** : l’assemblage **déclaratif** des étapes (pipeline de traitement)

On peut imaginer ce schéma très simple :

```text
              +---------------------------+
User input -> |   PromptTemplate          | -> texte formaté
              +---------------------------+
                           |
                           v
                     +-----------+
                     |   LLM     | -> réponse brute du modèle
                     +-----------+
                           |
                           v
                   +------------------+
                   | Output Parser    | -> string / JSON / etc.
                   +------------------+
```  
LangChain permet de **décrire ce pipeline en Python**, plutôt que de tout reprogrammer à la main.

## 🔧 Installation (à faire une fois dans votre environnement)

Décommentez et exécutez la cellule ci-dessous si besoin.

In [10]:
!pip install -U langchain-core langchain-ollama python-dotenv
# Pour LM Studio (API compatible OpenAI):
!pip install -U langchain-openai python-dotenv
# Optionnel : outils supplémentaires
# !pip install -U langchain-text-splitters

## ⚙️ Initialisation du LLM

On crée un objet LLM local qui sera la brique "modèle de langage" côté LangChain.
Vous pouvez utiliser soit **Ollama** soit **LM Studio** en modifiant le fichier `.env`.

In [11]:
import os
from dotenv import load_dotenv

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

# Configuration du LLM local
llm_type = os.getenv("LOCAL_LLM_TYPE", "ollama")

if llm_type == "ollama":
    from langchain_ollama import ChatOllama
    
    ollama_base_url = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")
    ollama_model = os.getenv("OLLAMA_MODEL", "llama3.2:latest")
    
    model = ChatOllama(
        model=ollama_model,
        base_url=ollama_base_url,
        temperature=0.2,
    )
    print(f"Utilisation d'Ollama avec le modèle {ollama_model}")
    
elif llm_type == "lmstudio":
    from langchain_openai import ChatOpenAI
    
    lmstudio_base_url = os.getenv("LMSTUDIO_BASE_URL", "http://localhost:1234/v1")
    lmstudio_model = os.getenv("LMSTUDIO_MODEL", "local-model")
    
    model = ChatOpenAI(
        model=lmstudio_model,
        base_url=lmstudio_base_url,
        api_key="not-needed",  # LM Studio ne nécessite pas de clé API
        temperature=0.2,
    )
    print(f"Utilisation de LM Studio avec le modèle {lmstudio_model}")
    
else:
    raise ValueError(f"LOCAL_LLM_TYPE non reconnu: {llm_type}. Utilisez 'ollama' ou 'lmstudio'.")

model

Utilisation de LM Studio avec le modèle mistralai/devstral-small-2-2512


ChatOpenAI(profile={}, client=<openai.resources.chat.completions.completions.Completions object at 0x11139dc60>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11139e140>, root_client=<openai.OpenAI object at 0x11139de10>, root_async_client=<openai.AsyncOpenAI object at 0x11139e020>, model_name='mistralai/devstral-small-2-2512', temperature=0.2, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='http://localhost:1234/v1')

## 1️⃣ Premier appel au LLM via LangChain

On commence par l’usage le plus simple : on envoie une liste de messages au modèle.

In [12]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Tu es un assistant pédagogique, clair et concis."),
    HumanMessage(content="Explique en deux phrases ce qu'est un modèle de langage."),
]

response = model.invoke(messages)
print(response.content)

Un modèle de langage est un algorithme d'intelligence artificielle entraîné sur d'énormes quantités de texte pour comprendre et générer du langage humain. Il utilise des techniques comme le *deep learning* (réseaux de neurones) pour prédire et produire des phrases cohérentes en fonction du contexte donné.


## 2️⃣ Séparer le **prompt** et le **modèle**

Plutôt que de construire les messages à la main, on décrit un **template de conversation**.

On utilise `ChatPromptTemplate` pour définir :
- le rôle système (persona),
- le message utilisateur avec une **variable `{question}`**.

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", f"Tu es un assistant IA qui vulgarise pour un public de développeurs curieux."),
        ("human", "Question : {question}"),
    ]
)


# LCEL: LangChain Expression Language
chain = prompt | model 

answer = chain.invoke({"question": "Quelle est la différence entre un LLM et LangChain ?"})
print(answer)



content='Ah, excellente question ! Je vais te l\'expliquer simplement avec des analogies et des concepts que tu connais déjà en tant que développeur.\n\n### **1. LLM (Large Language Model) : Le "Cerveau"**\nUn LLM, c\'est comme un modèle de langage pré-entraîné (ex: GPT-3, GPT-4, Llama) qui a appris à comprendre et générer du texte en analysant des milliards de données. C\'est un peu comme une **boîte noire** qui prend du texte en entrée et produit du texte en sortie.\n\n- **Exemple concret** :\n  Si tu lui donnes la prompt `"Écris un code Python pour calculer la factorielle d\'un nombre"`, il va te répondre avec une fonction comme `def factorial(n): ...`.\n\n- **Limites** :\n  - Il ne sait pas faire grand-chose *seul* : pas d\'accès à des bases de données, pas de mémoire entre les requêtes (sauf avec des techniques comme le *few-shot learning*), et pas de logique complexe sans guidage.\n\n---\n\n### **2. LangChain : Le "Framework" pour Super-Pouvoirs**\nLangChain, c\'est une **bibliot

In [14]:
parser = StrOutputParser()

parser.invoke(answer)

'Ah, excellente question ! Je vais te l\'expliquer simplement avec des analogies et des concepts que tu connais déjà en tant que développeur.\n\n### **1. LLM (Large Language Model) : Le "Cerveau"**\nUn LLM, c\'est comme un modèle de langage pré-entraîné (ex: GPT-3, GPT-4, Llama) qui a appris à comprendre et générer du texte en analysant des milliards de données. C\'est un peu comme une **boîte noire** qui prend du texte en entrée et produit du texte en sortie.\n\n- **Exemple concret** :\n  Si tu lui donnes la prompt `"Écris un code Python pour calculer la factorielle d\'un nombre"`, il va te répondre avec une fonction comme `def factorial(n): ...`.\n\n- **Limites** :\n  - Il ne sait pas faire grand-chose *seul* : pas d\'accès à des bases de données, pas de mémoire entre les requêtes (sauf avec des techniques comme le *few-shot learning*), et pas de logique complexe sans guidage.\n\n---\n\n### **2. LangChain : Le "Framework" pour Super-Pouvoirs**\nLangChain, c\'est une **bibliothèque** 

👉 Ici, on vient d’instancier le **schéma précédent** :

```text
variables -> PromptTemplate -> LLM -> OutputParser -> string
```

LangChain gère pour nous :
- la création des messages,
- l’appel modèle,
- la conversion de la réponse en `str`.

## 3️⃣ Ajouter un peu de "contexte" (mini-RAG sans base vectorielle)

On simule ici un RAG **très simplifié** :  
- un mini "corpus" en mémoire,  
- une fonction de récupération naïve (recherche par mots-clés),  
- un prompt qui combine **question + contexte**.

L’objectif est juste de montrer la **brique "retrieval"** dans le pipeline.

In [15]:
# Mini "base de connaissances" en mémoire
DOCUMENTS = [
    {
        "title": "Agents déterministes",
        "content": "Un agent déterministe suit un scénario fixe, sans appel à un modèle de langage à chaque étape."
    },
    {
        "title": "Agents IA (LLM)",
        "content": "Un agent IA utilise un modèle de langage pour décider dynamiquement des actions à chaque étape."
    },
    {
        "title": "LangChain",
        "content": "LangChain est un framework qui aide à structurer les appels aux LLM et à intégrer des données, des outils et de la mémoire."
    },
]

def simple_retriever(query: str, k: int = 2) -> str:
    """
    Très simple :
    - calcule un score basé sur le nombre de tokens communs
    - renvoie les k meilleurs documents concaténés
    """
    tokens = set(query.lower().split())
    scored = []
    for doc in DOCUMENTS:
        score = len(tokens.intersection(set(doc["content"].lower().split())))
        scored.append((score, doc))
    scored.sort(key=lambda x: x[0], reverse=True)
    top_docs = [doc["content"] for score, doc in scored[:k] if score > 0]
    if not top_docs:
        return "Aucun document pertinent trouvé dans la base locale."
    return "\n\n---\n\n".join(top_docs)

print(simple_retriever("Quelle différence entre agent IA et agent déterministe ?"))

Un agent déterministe suit un scénario fixe, sans appel à un modèle de langage à chaque étape.

---

Un agent IA utilise un modèle de langage pour décider dynamiquement des actions à chaque étape.


### 🔗 Chaîne RAG simplifiée 

On utilise `RunnableLambda` et `RunnablePassthrough` pour construire un pipeline :

```text
question
  ├──> (pass-through) ────────────────┐
  └──> retriever(question) -> contexte┤
                                      v
                           Prompt(question + contexte)
                                    |
                                    v
                                 LLM
                                    |
                                    v
                              OutputParser
```

In [16]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Étape 1 : préparer le "retriever" comme Runnable
retriever_runnable = RunnableLambda(lambda q: simple_retriever(q))

# Étape 2 : définir le prompt qui intègre question + contexte
rag_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Tu es un assistant qui répond en t'appuyant STRICTEMENT sur le contexte fourni.\n"
            "Si une information n'est pas dans le contexte, dis-le explicitement."
        ),
        (
            "human",
            "Contexte :\n{context}\n\n"
            "Question utilisateur : {question}"
        ),
    ]
)

rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": retriever_runnable,
    }
    | rag_prompt
    | model
    | StrOutputParser()
)

question = "Explique la différence entre un agent déterministe et un agent IA."
print(rag_chain.invoke(question))

Dans le contexte fourni, un **agent déterministe** suit un scénario fixe sans appel à un modèle de langage (LLM) à chaque étape. Cela signifie que ses actions sont préprogrammées et ne dépendent pas d'une intelligence artificielle pour prendre des décisions en temps réel.

En revanche, un **agent IA** (ou agent basé sur LLM) utilise un modèle de langage pour générer des réponses ou des actions dynamiques en fonction du contexte, des données d'entrée et éventuellement de la mémoire. LangChain, par exemple, est un framework qui facilite l'intégration de ces agents IA en structurant les appels aux LLM et en ajoutant des outils ou des données pour enrichir leurs capacités.

Si vous souhaitez une comparaison plus détaillée, il faudrait préciser le contexte des agents IA (par exemple, leur architecture ou leur utilisation dans LangChain).


## ✅ Récap
 
Briques présentées dans ce notebook :
  - modèle de langage (`ChatOpenAI`),
  - prompts (`ChatPromptTemplate`),
  - pipeline (`chain = prompt | model | parser`),
  - retrieval/RAG (même très simplifié).

Pour la suite nous allons aborder :  
- l’orchestration **multi-étapes** avec LangGraph,  
- puis le **pattern d’agent** plus avancé dans le dernier notebook.